In [1]:
%load_ext autoreload
%autoreload 2
%aimport -sys
%aimport -rpy2
%aimport -pandas
%aimport -numpy
%aimport -plotly

In [143]:
import sys

if '..' not in sys.path:
    sys.path.insert(0, '..')

import glm
import plotter
import plotly.express as px
from pandas import DataFrame
from numpy import ndarray
import pandas as pd
import numpy as np
from plotly.graph_objects import Figure as PlotlyFigure
import statsmodels.api as sm

In [133]:
N = 10
results = glm.compare_sklearn(features=N)

In [45]:
plot_df = results['dataframe'].copy()
for i in range(1, N+1):
    plot_df[f'feat_{i}_bin'] = \
        pd.qcut(
            x=plot_df[f'feat_{i}'], 
            q=10, 
            # include_lowest=True, 
            labels=False)

In [46]:
plot_df.corr().style.background_gradient(cmap='coolwarm')

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,y,feat_1_bin,feat_2_bin,feat_3_bin,feat_4_bin,feat_5_bin,feat_6_bin,feat_7_bin,feat_8_bin,feat_9_bin,feat_10_bin
feat_1,1.000000,0.000754,-0.000528,0.000694,-0.000399,0.000869,0.000793,0.000086,-0.000287,0.000030,0.440804,0.967122,0.000671,-0.000751,0.000804,-0.000100,0.000955,0.000748,-0.000140,-0.000394,0.000122
feat_2,0.000754,1.000000,0.000237,0.000097,-0.000462,0.001535,0.000531,-0.000859,0.000236,0.000149,0.194643,0.001043,0.967147,0.000321,0.000143,-0.000375,0.001552,0.000647,-0.000824,0.000186,0.000360
feat_3,-0.000528,0.000237,1.000000,0.000197,-0.000950,-0.000743,0.000183,-0.000123,-0.001006,0.000459,0.394109,-0.000400,-0.000139,0.967060,0.000280,-0.000900,-0.000743,-0.000086,-0.000196,-0.000931,0.000341
feat_4,0.000694,0.000097,0.000197,1.000000,0.000310,0.000601,-0.001409,0.000490,-0.000233,0.000191,0.426648,0.000505,0.000218,0.000409,0.967116,0.000401,0.000508,-0.001341,0.000363,-0.000317,0.000252
feat_5,-0.000399,-0.000462,-0.000950,0.000310,1.000000,0.000959,0.000642,0.000168,-0.000169,-0.000326,0.377305,-0.000523,-0.000308,-0.000841,0.000500,0.967155,0.000996,0.000643,0.000206,-0.000026,-0.000444
feat_6,0.000869,0.001535,-0.000743,0.000601,0.000959,1.000000,-0.000896,-0.000896,0.000533,0.000054,0.053749,0.000806,0.001409,-0.000607,0.000550,0.001078,0.967145,-0.000934,-0.001127,0.000708,-0.000186
feat_7,0.000793,0.000531,0.000183,-0.001409,0.000642,-0.000896,1.000000,-0.000130,-0.000800,0.000330,0.242053,0.000857,0.000504,0.000229,-0.001496,0.000647,-0.001037,0.967229,-0.000092,-0.000536,0.000282
feat_8,0.000086,-0.000859,-0.000123,0.000490,0.000168,-0.000896,-0.000130,1.000000,-0.000555,0.000752,0.093053,0.000073,-0.000958,-0.000146,0.000154,0.000165,-0.000836,-0.000117,0.967103,-0.000462,0.001052
feat_9,-0.000287,0.000236,-0.001006,-0.000233,-0.000169,0.000533,-0.000800,-0.000555,1.000000,-0.000112,0.446607,-0.000302,0.000207,-0.000944,-0.000174,-0.000013,0.000577,-0.000712,-0.000556,0.967194,-0.000038
feat_10,0.000030,0.000149,0.000459,0.000191,-0.000326,0.000054,0.000330,0.000752,-0.000112,1.000000,0.133774,0.000081,0.000337,0.000320,0.000222,-0.000456,0.000136,0.000216,0.000940,-0.000063,0.967209


In [47]:
boxplots = []
for i in range(1, N+1):
    fig: PlotlyFigure = \
        plotter.boxplot(
            data=plot_df.sample(100000),
            x=f'feat_{i}_bin',
            y='y',
            points = False)
    boxplots.append(fig)

plotter.save_plots_to_html(
    figures=boxplots, 
    file='./exhibits/boxplots.html')
del(boxplots)


In [48]:
scatterplots = [] 
for i in range(1, N+1):
    fig: PlotlyFigure = \
        px.scatter(
            data_frame=plot_df.sample(50000), 
            x=f'feat_{i}', 
            y='y',
            render_mode='webgl')
    scatterplots.append(fig)

plotter.save_plots_to_html( #can only render 8 at most
    figures=scatterplots, 
    file='./exhibits/scatterplots.html')
del(scatterplots)

In [49]:
for i in range(1, N+1):
    fig = px.histogram(plot_df.sample(25000), x=f'feat_{i}', histnorm='probability density')
    # fig.show()

In [201]:
# create a q-q plot
from scipy.stats import norm
# from statsmodels.discrete.discrete_model import Logit as LogisticRegressor
from statsmodels.formula.api import logit as LogisticRegressor
from statsmodels.discrete.discrete_model import Logit

In [186]:
data = sm.datasets.get_rdataset("Duncan", "carData").data
data['prestige_ge_60'] = np.where(data['prestige'] >= 60, 1, 0)
data.head()

,type,income,education,prestige,prestige_ge_60
accountant,prof,62,86,82,1
pilot,prof,72,76,83,1
architect,prof,75,92,90,1
author,prof,55,90,76,1
chemist,prof,64,86,90,1


In [189]:
predictors = [
    'income',
    'education'
]
formula = '+'.join(predictors)
model: Logit = (
    LogisticRegressor(
        formula=f'prestige_ge_60 ~ {formula}',
        data=data)
    .fit() 
)

Optimization terminated successfully.
         Current function value: 0.268184
         Iterations 8


method

In [197]:
residuals = model.resid_response.values
sorted_residuals = np.array(sorted(residuals))

In [198]:
# sorted_residuals = np.array(sorted(results['true_y'] - results['fitted_y'])) 
sample_size = len(sorted_residuals)
probabilities = np.array([1.0, *np.arange(start=5, stop=96, step=1), 99.0]) / 100
observed_quantiles = np.quantile(sorted_residuals, probabilities)
theoretical_quantiles = norm.ppf(probabilities, loc=10, scale=50)

In [199]:
px.scatter(
    x=observed_quantiles,
    y=theoretical_quantiles,
    title='qq-plot (Standard Normal)')